# Qianle_Tang_HW1

In [19]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import bisect, brentq
import math

# Problem 1

In [4]:
class UpAndOutPut:
    
    def __init__(self, K, T, barrier, observationinterval):
        self.K = K
        self.T = T
        self.barrier = barrier
        self.observationinterval = observationinterval

In [5]:
hw1contract = UpAndOutPut(K=95, T=0.25, barrier=114, observationinterval=0.02)

In [6]:
class GBMdynamics: 
    
    def __init__(self, S, r, rGrow, sigma=None):
        self.S = S
        self.r = r
        self.rGrow = rGrow
        self.sigma = sigma
        
    def update_sigma(self, sigma):
        self.sigma = sigma
        return self

In [7]:
hw1dynamics = GBMdynamics(S=100, sigma=0.4, rGrow=0, r=0)

In [15]:
class Tree:
    
    def __init__(self, N):
        self.N = N
                
    def price_upandout(self, dynamics, contract): 
        
        deltat = contract.T / self.N
        J = np.ceil(np.log(contract.barrier/dynamics.S)/(dynamics.sigma*np.sqrt(3*deltat))-0.5)
        deltax = np.log(contract.barrier/dynamics.S)/(J+0.5)
        
        Sgrid = dynamics.S*np.exp(np.linspace(self.N, -self.N, num=2*self.N+1, endpoint=True)*deltax)  
        #Here I decided to make the SMALLER indexes in this array correspond to HIGHER S
        
        numTimestepsPerObs = contract.observationinterval/deltat
        if abs(numTimestepsPerObs-round(numTimestepsPerObs)) > 1e-8:
            raise ValueError("This value of N fails to place the observation dates in the tree.")
            
        nu = 0        # complete this 
        Pu = 0.5*((dynamics.sigma**2*deltat + ((dynamics.rGrow-dynamics.sigma**2/2)**2)*(deltat**2))/deltax**2\
                  +(dynamics.rGrow-dynamics.sigma**2/2)*deltat/deltax)       
        Pd = 0.5*((dynamics.sigma**2*deltat + ((dynamics.rGrow-dynamics.sigma**2/2)**2)*(deltat**2))/deltax**2\
                  -(dynamics.rGrow-dynamics.sigma**2/2)*deltat/deltax)   
        Pm = 1 - ((dynamics.sigma**2*deltat + ((dynamics.rGrow-dynamics.sigma**2/2)**2)*(deltat**2))/deltax**2)

        optionprice = np.maximum(contract.K-Sgrid,0)   #an array of time-T option prices.
        
        #Next, induct backwards to time 0, updating the optionprice array 
        #Hint: if x is an array, then what are x[2:] and x[1:-1] and x[:-2]
    
        for t in np.linspace(self.N-1, 0, num=self.N, endpoint=True)*deltat:
            cur_price = []
            for i in range(1,len(optionprice)-1):
                #if t % contract.observationinterval == 0 and Sgrid[i] >= contract.barrier:
                if abs(t % contract.observationinterval) < 1e-8 and Sgrid[i] >= contract.barrier:
                    # check if its the observation date:
                    cur_price.append(0)
                else:
                    cur_price.append(np.exp(-dynamics.r*deltat) * (Pu*optionprice[i-1] + Pm*optionprice[i] + Pd*optionprice[i+1]))
            optionprice = cur_price
            Sgrid = np.delete(Sgrid, [0,-1])
                    
        return optionprice[0]         
    def price_upandout_continuous(self, dynamics, contract): 
        
        deltat = contract.T / self.N
        J = np.ceil(np.log(contract.barrier/dynamics.S)/(dynamics.sigma*np.sqrt(3*deltat))-0.5)
        deltax = np.log(contract.barrier/dynamics.S)/(J+0.5)
        
        Sgrid = dynamics.S*np.exp(np.linspace(self.N, -self.N, num=2*self.N+1, endpoint=True)*deltax)  
        #Here I decided to make the SMALLER indexes in this array correspond to HIGHER S
        
        numTimestepsPerObs = contract.observationinterval/deltat
        if abs(numTimestepsPerObs-round(numTimestepsPerObs)) > 1e-8:
            raise ValueError("This value of N fails to place the observation dates in the tree.")
            
        nu = 0        # complete this 
        Pu = 0.5*((dynamics.sigma**2*deltat + ((dynamics.rGrow-dynamics.sigma**2/2)**2)*(deltat**2))/deltax**2\
                  +(dynamics.rGrow-dynamics.sigma**2/2)*deltat/deltax)       
        Pd = 0.5*((dynamics.sigma**2*deltat + ((dynamics.rGrow-dynamics.sigma**2/2)**2)*(deltat**2))/deltax**2\
                  -(dynamics.rGrow-dynamics.sigma**2/2)*deltat/deltax)   
        Pm = 1 - ((dynamics.sigma**2*deltat + ((dynamics.rGrow-dynamics.sigma**2/2)**2)*(deltat**2))/deltax**2)

        optionprice = np.maximum(contract.K-Sgrid,0)   #an array of time-T option prices.
        
        #Next, induct backwards to time 0, updating the optionprice array 
        #Hint: if x is an array, then what are x[2:] and x[1:-1] and x[:-2]
    
        for t in np.linspace(self.N-1, 0, num=self.N, endpoint=True)*deltat:
            cur_price = []
            for i in range(1,len(optionprice)-1):
                if Sgrid[i] >= contract.barrier:
                    cur_price.append(0)
                else:
                    cur_price.append(np.exp(-dynamics.r*deltat) * (Pu*optionprice[i-1] + Pm*optionprice[i] + Pd*optionprice[i+1]))
            optionprice = cur_price
            Sgrid = np.delete(Sgrid, [0,-1])
                    
        return optionprice[0] 
        

#### a) Write a Python function to price our option at time 0 using a trinomial tree.

In [16]:
hw1tree=Tree(N=10000)

hw1tree.price_upandout(hw1dynamics, hw1contract)

5.30114218920439

#### b) Using your part (a) result, find the time-0 price of the up-and-in put.

In-Out Partiry for Barrier Options: 
    
- $UI(K,T,B) = Vanilla(K,T) - UO(K,T,B)$

In [17]:
# BS function for put option
def BSput(sigma,S,rGrow,r,K,T):
    F=S*np.exp(rGrow*T)
    sd = sigma*np.sqrt(T)
    d1 = np.log(F/K)/sd+sd/2
    d2 = d1-sd
    return np.exp(-r*T)*K*norm.cdf(-d2)-F*norm.cdf(-d1)

vanilla_put = BSput(0.4, 100, 0, 0, 95, 0.25)
vanilla_put

5.519541063676975

In [18]:
print("The time-0 price of the up-and-in put is: ", vanilla_put-5.30114218920439)

The time-0 price of the up-and-in put is:  0.21839887447258466


#### c) Consider a continuously-monitored barrier option

##### c1)

time-0 price of the continuously-monitored barrier option should be smaller than the time-0 price of the discretely-monitored option. Intuitionally, this option is putting a stricter condition on the pay-off, so the price should be lower.

##### c2)

$(114-95) - \alpha (136.8-114) = 0$

$\therefore  \alpha = 0.833$

Based on the BSM, the time-0 price of the vanilla put is 5.52, and the time-0 price of the vanilla calls is 0.58

so the the time-0 value of the continuously-monitored barrier option is $ 5.52 - 0.58*0.833 = 5.04$

# Problem 2

In [ ]:
# uses the same GBMdynamics class as in Problem 1

In [124]:
class CallOption:
    
    def __init__(self, K, T, price=None):
        self.K = K
        self.T = T
        self.price = price

    def BSprice(self, dynamics):
        # ignores self.price if given, because this function calculates price based on the dynamics 
        
        F = dynamics.S*np.exp(dynamics.rGrow*self.T)
        sd = dynamics.sigma*np.sqrt(self.T)
        d1 = np.log(F/self.K)/sd+sd/2
        d2 = d1-sd
        return np.exp(-dynamics.r*self.T)*(F*norm.cdf(d1)-self.K*norm.cdf(d2))
    
    def IV(self, dynamics):
        # ignores dynamics.sigma, because this function solves for sigma.  
        
        if self.price is None: 
            raise ValueError('Contract price must be given')
    
        df = np.exp(-dynamics.r*self.T)  #discount factor
        F = dynamics.S / df
        lowerbound = np.max([0,(F-self.K)*df])
        C = self.price
        if C<lowerbound:
            return np.nan
        if C==lowerbound:
            return 0
        if C>=F*df:
            return np.nan 
        # C should be np.max([0,(F-self.K)*df]) < C < F*df

        dytry = dynamics
        # We "try" values of sigma until we find sigma that generates price C

        # First find lower and upper bounds
        dytry.sigma = 0.2
        while self.BSprice(dytry)>C:
            dytry.sigma /= 2
        while self.BSprice(dytry)<C:
            dytry.sigma *= 2
        hi = dytry.sigma
        lo = hi/2
                           
        # We have calculated "lo" and "hi" which bound the implied volatility from below and above. 
        # In other words, the implied volatility is somewhere in the interval [lo,hi].
        # Then, to calculate the implied volatility within that interval, 
        # for purposes of this homework, you may either (A) write your own bisection algorithm, 
        # or (B) use scipy.optimize.bisect or (C) use scipy.optimize.brentq
        # You will need to provide lo and hi to those solvers.
        # There are other solvers that do not require you to bound the solution 
        # from below and above (for instance, scipy.optimize.fsolve is a useful solver).  
        # However, if you are able to bound the solution (of a single-variable problem), 
        # then bisection or Brent will be more reliable.
        function = lambda sigma: self.BSprice(dynamics.update_sigma(sigma)) - self.price
        impliedVolatility = brentq(function , a=lo, b=hi, xtol=1e-4)     # you fill this in, using bisect or brentq imported from scipy.optimize,
                                 # or by writing your own bisection algorithm.
        return impliedVolatility


#### a) Find the time-0 Black-Scholes implied volatilities of these two options, by completing the coding of the IV function.

In [125]:
#European calls on S at 0.5-year with C = 11.25
dynamics2 = GBMdynamics(sigma=0.4, rGrow=0, S=100, r=0)
contract1 = CallOption(K=100, T=0.5)
contract1.price = 11.25
print("The implied volality of European calls on S at 0.5-year with C = 11.25: ",contract1.IV(dynamics2))

The implied volality of European calls on S at 0.5-year with C = 11.25:  0.40013455766201506


In [126]:
#European calls on S at 1-year with C = 12
contract2 = CallOption(K=100, T=1)
contract2.price = 12
print("The implied volality of European calls on S at 1-year with C = 12: ",contract2.IV(dynamics2))

The implied volality of European calls on S at 1-year with C = 12:  0.3019382674808541


#### b) Under that assumption, what would be the time-0 price of the 0.75-expiry call?

In [127]:
# The implied volality of this option is 
implied_volality = (0.40013455766201506+ 0.3019382674808541)/ 2
implied_volality

0.3510364125714346

In [128]:
#The Price 
dynamics3 = GBMdynamics(sigma=implied_volality, rGrow=0, S=100, r=0)
contract3 = CallOption(K=100, T=0.75)
print("The time-0 price of the 0.75-expiry call is: ",contract3.BSprice(dynamics3))

The time-0 price of the 0.75-expiry call is:  12.081560834203216


#### c) Arbitrage

We can long one share of contract2(1T) and short one share of contract3(0.75T)

- The time-0 value of your portfolio is -12.08+12 = -0.08

- The time-1 value of your portfolio is always greater than 0 since the risk-free interest rate is 0, and
    - - at time 0.75, you settle your 0.75T option(short), so you get -1 share of stock and K dollars 
    - - at time 1, you always settle your 1T option, so you get 1 share of stock and −K dollars
    - - two payoffs cancell out.